In [ ]:
import matplotlib.pyplot as plt
import datetime
import tilemapbase as tmb
import numpy as np
import pandas as pd
from pluma.stream.georeference import Georeference
from pluma.stream.ubx import _UBX_MSGIDS
from pluma.schema import Dataset

## Notebook plotting
%matplotlib inline
plt.style.use('ggplot')

## Figure export parameters
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
import matplotlib as mpl
mpl.rcParams.update(new_rc_params)


In [ ]:
stream_root_folder = r's3://emotional-cities/data/nrg/poc-v1/TechnicalBenchmarks/BenchmarkRun2022_09_22_10_30_02/'
from benchmark_schemas import build_benchmarkschema_tinkerforge

dataset = Dataset(
    stream_root_folder,
    datasetlabel="TinkerForge_Benchmark",
    georeference= Georeference(),
    schema=build_benchmarkschema_tinkerforge)
dataset.populate_streams(autoload=True)




In [ ]:
# Get the synch pulse from the BioData "Set" stream
bit_mask = 0x03
rising_edge_events = dataset.streams.BioData.Set.data[dataset.streams.BioData.Set.data & bit_mask > 0]
rising_edge_events

In [ ]:
# Get the raw voltage trace that records the synch pulse in the TK device

threshold = 20
ttl = (dataset.streams.TK.AnalogIn.Voltage.data["Value"].values > threshold).astype(int)
tk_rising_edge_events = dataset.streams.TK.AnalogIn.Voltage.data.iloc[np.where(np.diff(ttl, prepend=np.NaN) == 1)[0],:]
tk_rising_edge_events

print(f"Set events: {len(rising_edge_events)}\nADC events: {len(tk_rising_edge_events)}")


fig, axs = plt.subplots(1,2)
fig.set_size_inches(15,7)
axs[0].plot(dataset.streams.TK.AnalogIn.Voltage.data)
axs[0].scatter(x = rising_edge_events.index.values, y = np.ones(shape=rising_edge_events.index.values.shape) * 1000.0, c = 'blue')
axs[0].set_xlabel('Time (s)')
axs[0].set_ylabel('Voltage (adc units)')
axs[0].set_title("Raw voltage trace")
axs[0].set_xlim((rising_edge_events.index[0], rising_edge_events.index[2]))

axs[1].plot(dataset.streams.TK.AnalogIn.Voltage.data.index.values, ttl)
axs[1].scatter(x = rising_edge_events.index.values, y = np.ones(shape=rising_edge_events.index.values.shape), c = 'blue')
axs[1].set_xlabel('Time (s)')
axs[1].set_ylabel('Voltage (adc units)')
axs[1].set_title("Raw voltage trace")
axs[1].set_xlim((rising_edge_events.index[0], rising_edge_events.index[2]))
plt.show()


delta_t = rising_edge_events.index.values - tk_rising_edge_events.index.values
delta_t = -delta_t / np.timedelta64(1, 'ms')

plt.figure(figsize= (9,6))
plt.hist(delta_t, bins = 50, color= "Teal")
plt.xlabel('Latency (ms)')
plt.ylabel('Counts')
plt.title(f'TinkerForge latency benchmark\n$\mu = {np.mean(delta_t):.2f}, \sigma = {np.std(delta_t):.2f},$ [min:max] = [{np.min(delta_t):.2f}:{np.max(delta_t):.2f}]')
plt.show()
